In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:

def aa_masses(filename):
    mass_table = {}
    with open (filename, "r") as myfile:
        for line in myfile:
            aa_mass = line.rstrip().split(" ")
            mass_table[aa_mass[0]] = int(aa_mass[1])
    return(mass_table)


def score_peptides(experimental_spectrum, theoretical_spectrum):
    score = 0
    for i in experimental_spectrum:
        if i in theoretical_spectrum:
            score += 1
            theoretical_spectrum.remove(i)
    return(score)


def spectrum(peptides):
    spec = [0]
    for pep in peptides:
        spec.append(sum(pep))
    return(sorted(spec))


def linear_subpeptides(pep_seq):
    n = len(pep_seq)
    sub_peptides = []
    for size in range(1,n):
        for start in range(n-size+1):
            sub_peptides.append(pep_seq[start:start+size])
    sub_peptides.append(pep_seq)
    return(sub_peptides)


def cyclic_subpeptides(pep_seq):
    n = len(pep_seq)
    cyclo_seq = pep_seq + pep_seq
    sub_peptides = []
    for size in range(1,n):
        for start in range(n):
            sub_peptides.append(cyclo_seq[start:start+size])

    sub_peptides.append(pep_seq)
    return(sub_peptides)


def aa2masses(peptide, masses):
    pep_masses = []
    for i in peptide:
        pep_masses.append(masses[i])
    return(pep_masses)


def trim(peptide_leaderboard, exp_spectrum, trim_length):
    scores = []
    for lb in peptide_leaderboard:
        #changed this to linear
        score = score_peptides(exp_spectrum, spectrum(linear_subpeptides(lb)))
        scores.append(score)
    if len(scores) > trim_length:
        cutoff = sorted(scores, reverse = True)[trim_length-1]
        num_keep = len([i for i in scores if i >= cutoff])
        lb_sorted = [x for s,x in sorted(zip(scores, peptide_leaderboard), reverse = True)]
        return(lb_sorted[0:num_keep])
    else:
        return(peptide_leaderboard)

In [3]:
from itertools import combinations

def convolve(spectrum):
    pairs = list(combinations(spectrum, 2))
    conv = [abs(pair[1]-pair[0]) for pair in pairs]
    
    return(conv)

def choose_alphabet(spectrum, M):
    conv = [i for i in convolve(spectrum) if (i > 56) and (i < 201)]
    counts = [conv.count(i) for i in set(conv)]
    conv_sorted = [x for s,x in sorted(zip(counts, set(conv)), reverse = True)]
    if len(conv_sorted) > M:
        return(conv_sorted[0:M])
    else:
        return(conv_sorted)


In [4]:
def cyclic_leaderboard_sequence(exp_spec, trim_length, mass_set):

    parent_mass = max(exp_spec)
    peptides = [[]]
    leader_peptide = []

    while len(peptides) != 0:
        new_peptides = []
        
        # expand peptide list
        for pep in peptides:
            for mass in mass_set:
                new_pep = []
                new_pep.extend(pep)
                new_pep.append(mass)
                new_peptides.append(new_pep)
        
        # test each peptide to see if it's the leader peptide or greater than parent mass
        filtered_peptides = []
        for pep in new_peptides:

            if sum(pep) == parent_mass:
                new_spec = spectrum(cyclic_subpeptides(pep))
                leader_spec = spectrum(cyclic_subpeptides(leader_peptide))
                if score_peptides(exp_spec, new_spec) > score_peptides(exp_spec, leader_spec):
                    leader_peptide = pep
            if sum(pep) <= parent_mass:
                filtered_peptides.append(pep)

        # trim the list to just the top N peptides (and ties)
        peptides = trim(filtered_peptides, exp_spec, trim_length)
    
    return(leader_peptide)

In [6]:
f = open("dataset_104_7.txt", "r")
content = f.readlines()
f.close()

m = int(content[0])
n = int(content[1])
spec = [int(i) for i in content[2].split(" ")]

masses = choose_alphabet(spec, m)
sequence = cyclic_leaderboard_sequence(spec, n, masses)

sequence = [str(i) for i in sequence]

f = open("conv-sequencing-solution.txt", "w")
f.write("-".join(sequence))
f.close()
